<a href="https://colab.research.google.com/github/graylan0/compassion-mapping-llm/blob/main/Pennylane_Openai_Textblob_RGB_Quantum_States_GPT_3_5_Turbo_Compassion_Text_To_Report_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions
Please add your openai api key in the google secrets on the left hand side. Then when the script is complete. download your database and report.md file from the file symbol on the left. Repeat. Key symbol = where you put your "openai api key" google around to find how to get to that on openai's website. Thank you

In [23]:
!pip install openai==0.27.9
!pip install pennylane
!pip install aiosqlite
!pip install httpx
!pip install randomcolor

In [ ]:
!wget https://github.com/graylan0/compassion-mapping-llm/blob/main/user_input.json

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [60]:
import os
import logging
import json
import asyncio
import openai
import numpy as np
import pennylane as qml
import aiosqlite
from textblob import TextBlob
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
import string
from google.colab import userdata
import nest_asyncio

from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
nest_asyncio.apply()

# Retrieve OpenAI API key
OPENAI_API_KEY = userdata.get('openai_api_key')

if OPENAI_API_KEY is None:
   raise Exception("OpenAI API key not found. Please add it as a secret.")


# Define constants
GPT_MODEL = "gpt-3.5-turbo"
MAX_TOKENS = 50
TEMPERATURE = 0.7

# Initialize logger
logging.basicConfig(level=logging.ERROR)

# Load NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Initialize QML device
qml_model = qml.device("default.qubit", wires=4)

@qml.qnode(qml_model)
def quantum_circuit(color_code, amplitude):
 try:
     print(f"Received color code: {color_code}") # Debugging line

     # Check if color_code is a valid hexadecimal string
     if not all(c in '0123456789abcdefABCDEF' for c in color_code):
         logging.error(f"Invalid color code: {color_code}")
         return [0]*4 # Return a default quantum state

     r, g, b = [int(color_code[i:i+2], 16) for i in (0, 2, 4)]
     r, g, b = r / 255.0, g / 255.0, b / 255.0
     qml.RY(r * np.pi, wires=0)
     qml.RY(g * np.pi, wires=1)
     qml.RY(b * np.pi, wires=2)
     qml.RY(amplitude * np.pi, wires=3)
     qml.CNOT(wires=[0, 1])
     qml.CNOT(wires=[1, 2])
     qml.CNOT(wires=[2, 3])
     return qml.probs(wires=[0,1,2,3])
 except Exception as e:
     logging.error(f"Quantum circuit error: {e}")
     return [0]*4 # Return a default quantum state
async def create_db_connection():

   try:
       return await aiosqlite.connect('compassiondb.db')
   except Exception as e:
       logging.error(f"Database connection error: {e}")
       raise

async def sentiment_to_amplitude(text):
   try:
       analysis = TextBlob(text)
       return (analysis.sentiment.polarity + 1) / 2
   except Exception as e:
       logging.error(f"Sentiment analysis error: {e}")
       return 0.5

import httpx
import json

async def generate_html_color_codes(sentence):
 try:
    async with httpx.AsyncClient() as client:
        data = {
            "model": GPT_MODEL,
            "messages": [
              {"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": f"Generate HTML color code for the sentence: PLZ ONLY REPLY WITH THE HTML COLOR CODE ONLY THAT ONE ITEM[itemhtmlcolorcodedescribeingtheemotionstate] THAT PLEASE FOR THIS SENTENCE '{sentence}'"},
            ],
            "temperature": TEMPERATURE,
            "max_tokens": MAX_TOKENS
        }
        headers = {"Authorization": f"Bearer {OPENAI_API_KEY}"}
        response = await client.post("https://api.openai.com/v1/chat/completions", json=data, headers=headers)
        response_text = response.json()['choices'][0]['message']['content'].strip()

        print(f"Response from OpenAI API: {response_text}") # Debugging line

        match = re.search(r'#[0-9a-fA-F]{3}(?:[0-9a-fA-F]{3})?', response_text)
        if match:
            color_code = match.group(0)[1:] # Strip the '#' from the color code
            print(f"Generated color code: {color_code}") # Debugging line
            return color_code
        else:
            logging.error(f"No valid color code found in response: {response_text}")
            return "#FFFFFF" # Return a default color code
 except Exception as e:
    logging.error(f"Error in generating HTML color codes: {e}")
    return "#FFFFFF" # Return a default color code

async def extract_user_details(user_input):
   try:
       blob = TextBlob(user_input)
       pos_tags = blob.tags
       sentences = sent_tokenize(user_input)
       words = [word_tokenize(sentence) for sentence in sentences]
       named_entities = [pos_tag(word) for word in words]
       return pos_tags, named_entities
   except Exception as e:
       logging.error(f"Error in extracting user details: {e}")
       return [], []

async def generate_dynamic_values_prompt(user_input):
  ...
  return "", 0, 0

async def calculate_narcissism_score(user_input):
  # Simulate narcissism calculation
  return len(user_input) * 0.01

async def generate_dynamic_compassion_prompt(scenario_type, user_input, quantum_state):
  system_message = ""
  user_message = ""
  if scenario_type == "relationship":
      system_message = (
          "You are an advanced AI system exploring the dynamics of trust and communication in long-term romantic relationships. "
          "Consider challenges such as miscommunication, emotional support, and maintaining intimacy over time. "
          f"Provide insights into fostering a healthy and compassionate connection between partners using advanced AI empathy from the datasource: {user_input}."
          f"\n\nQuantum State: {quantum_state}"
      )
      user_message = f"System, generate an active compassion scenario guideplan exploring the dynamics of trust and communication in a long-term romantic relationship using advanced AI empathy from the datasource: {user_input}"
  elif scenario_type == "corporate":
      system_message = (
          "You are an advanced AI system providing scenarios for a corporate environment transitioning to a more empathetic and employee-centric culture. "
          "Address challenges such as balancing productivity with employee well-being, fostering a sense of belonging, and implementing policies that support mental health. "
          f"Provide insights on how a company can navigate this transformation while ensuring both organizational success and employee satisfaction using advanced AI empathy from the datasource: {user_input}."
          f"\n\nQuantum State: {quantum_state}"
      )
      user_message = f"System, create an active compassion scenario guideplan transvocating a corporate environment transitioning to a more empathetic and employee-centric culture. advanced AI empathy from the datasource: {user_input}"
  elif scenario_type == "work_life_balance":
      system_message = (
          "You are an advanced AI system creating compassion scenarios for a user struggling with balancing work and personal life. "
          f"Consider factors such as job stress, family responsibilities, and other challenges. Provide insights into fostering a healthy balance using advanced AI empathy from the datasource: {user_input}."
          f"\n\nQuantum State: {quantum_state}"
      )
      user_message = f"System, dynamically generate an active compassion scenario guideplan involving a user struggling with balancing work and personal life. using advanced AI empathy from the datasource: {user_input}."
  else:
      return "Invalid scenario type."

  async with httpx.AsyncClient() as client:
      response = await client.post(
          "https://api.openai.com/v1/chat/completions",
          json={
              "model": "gpt-3.5-turbo",
              "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message},
              ],
              "temperature": 0.7,
              "max_tokens": 300,
          },
          headers={"Authorization": f"Bearer {OPENAI_API_KEY}"},
      )
  return response.json()['choices'][0]['message']['content']

async def process_user_input(user_input, db_conn):
    try:
        sentiment_amplitude = await sentiment_to_amplitude(user_input)
        logging.info(f"Sentiment amplitude calculated: {sentiment_amplitude}")

        color_code = await generate_html_color_codes(user_input)
        logging.info(f"Color code generated: {color_code}")

        amplitude = await sentiment_to_amplitude(user_input)
        logging.info(f"Amplitude calculated: {amplitude}")

        try:
            quantum_state = quantum_circuit(color_code, amplitude)
            logging.info(f"Quantum state generated: {quantum_state}")
        except Exception as e:
            logging.error(f"Error in quantum circuit: {e}")
            quantum_state = "Error"

        try:
            color_code, amplitude, narcissism_score = await generate_dynamic_values_prompt(user_input)
            logging.info(f"Dynamic values generated - Color Code: {color_code}, Amplitude: {amplitude}, Narcissism Score: {narcissism_score}")
        except Exception as e:
            logging.error(f"Error in generating dynamic values: {e}")
            color_code, amplitude, narcissism_score = "Error", 0, 0

        try:
            scenario_type = "relationship"
            compassion_scenario = await generate_dynamic_compassion_prompt(scenario_type, user_input, quantum_state)
            logging.info(f"Compassion scenario generated: {compassion_scenario}")
        except Exception as e:
            logging.error(f"Error in generating compassion scenario: {e}")
            compassion_scenario = "Error in generating compassion scenario."

        markdown_output = f"## User Input\n\n- **Input**: {user_input}\n\n"
        markdown_output += f"## Sentiment Amplitude\n\n- **Amplitude**: {sentiment_amplitude}\n\n"
        markdown_output += f"## Quantum State: {quantum_state}\n"
        markdown_output += f"## Compassion Scenario\n\n{compassion_scenario}\n\n"

        return markdown_output
    except Exception as e:
        logging.error(f"General error processing user input: {e}")
        return f"Error processing user input: {e}"

async def generate_dynamic_compassion_prompt(scenario_type, user_input, quantum_state):
    try:
        async with httpx.AsyncClient() as client:
            response = await client.post(
                "https://api.openai.com/v1/chat/completions",
                json={
                    "model": "gpt-3.5-turbo",
                    "messages": [
                        {"role": "system", "content": f"You are an advanced AI system. Scenario type: {scenario_type}."},
                        {"role": "user", "content": user_input},
                    ],
                    "temperature": 0.7,
                    "max_tokens": 300,
                },
                headers={"Authorization": f"Bearer {OPENAI_API_KEY}"},
            )
            if response.status_code != 200:
                logging.error(f"OpenAI API error: {response.text}")
                return "Error in OpenAI API call."

            response_content = response.json()['choices'][0]['message']['content']
            logging.info(f"Compassion scenario response: {response_content}")
            return response_content
    except Exception as e:
        logging.error(f"Error in generating compassion scenario: {e}")
        return "Error in generating compassion scenario."

async def main():
 try:
  db_conn = await create_db_connection()
  markdown_content = "# Compassion Analysis Report\n\n"

  with open('user_input.json') as json_file:
      user_inputs = json.load(json_file)

  tasks = [process_user_input(user_input, db_conn) for user_input in user_inputs]
  for task in asyncio.as_completed(tasks):
      try:
         data = await task
         markdown_content += data
      except Exception as exc:
         print('%r generated an exception: %s' % (user_inputs, exc))

  with open('report.md', 'w') as md_file:
      md_file.write(markdown_content)
 except Exception as e:
  logging.error(f"An error occurred in main: {e}")
 finally:
  if db_conn:
      await db_conn.close()

if __name__ == "__main__":
 asyncio.run(main())





SecretNotFoundError: Secret openai_api_key does not exist.

# Sample Output
# Compassion Analysis Report

## User Input

- **Input**: I'm so excited about the upcoming event, I've been looking forward to it for weeks!

## Sentiment Amplitude

- **Amplitude**: 0.734375

## Quantum State: [6.15728338e-34 3.13367112e-33 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.35779477e-01 1.64220523e-01]
## Compassion Scenario

That's great to hear! It sounds like you have something special planned. What event are you excited about?

## User Input

- **Input**: I'm feeling incredibly happy today, everything seems to be going my way and nothing can bring me down!

## Sentiment Amplitude

- **Amplitude**: 0.6513888888888889

## Quantum State: [6.04650590e-35 1.62601348e-34 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.57047448e-33 9.55858565e-34
 2.54936444e-01 6.85569653e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.33673045e-02 1.61265983e-02]
## Compassion Scenario

That's wonderful to hear! It's great to experience such joy and positivity in life. Enjoy this moment and make the most of it. Is there anything specific that has contributed to your happiness today?

## User Input

- **Input**: Today is a sad day, I feel like nothing is going right and I'm struggling to find my way.

## Sentiment Amplitude

- **Amplitude**: 0.4464285714285714

## Quantum State: [0.07162897 0.05107525 0.05170839 0.0725169  0.07341584 0.05234938
 0.05170839 0.0725169  0.07341584 0.05234938 0.05299832 0.07432592
 0.07341584 0.05234938 0.05170839 0.0725169 ]
## Compassion Scenario

I'm sorry to hear that you're feeling down today. It's completely normal to have days where everything feels overwhelming and nothing seems to go right. Remember that it's okay to feel sad and take some time to process your emotions. Is there anything specific that's been bothering you or causing you to feel this way?

## User Input

- **Input**: I'm feeling very peaceful and content today, everything is in balance and it's a good day.

## Sentiment Amplitude

- **Amplitude**: 0.75625

## Quantum State: [0.06935672 0.42756331 0.         0.         0.         0.
 0.43286349 0.07021648 0.         0.         0.         0.
 0.         0.         0.         0.        ]
## Compassion Scenario

That's wonderful to hear! Feeling peaceful and content is a great state to be in. It's important to cherish those moments of balance and happiness. Is there anything specific that contributed to this feeling of peace and contentment today?

## User Input

- **Input**: I'm in love! Everything feels perfect and I'm so grateful for the wonderful people in my life.

## Sentiment Amplitude

- **Amplitude**: 0.9375

## Quantum State: [1.35182084e-35 1.39354978e-33 2.26375015e-33 2.19596364e-35
 3.36708972e-37 3.47102737e-35 2.13674173e-35 2.07275840e-37
 5.52824105e-05 5.69889060e-03 9.25755553e-03 8.98034408e-05
 5.85684097e-03 6.03763398e-01 3.71672796e-01 3.60543297e-03]
## Compassion Scenario

That's wonderful to hear! Love can bring so much joy and fulfillment to our lives. Cherishing the people who bring happiness and positivity is a great way to appreciate the beauty of relationships. What makes you feel grateful for the wonderful people in your life?

## User Input

- **Input**: I'm feeling very anxious today, it's hard to focus and I feel like I'm on edge all the time.

## Sentiment Amplitude

- **Amplitude**: 0.3458333333333333

## Quantum State: [2.74758714e-33 1.00181231e-33 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.67192740e-01 7.32807260e-01]
## Compassion Scenario

I'm sorry to hear that you're feeling anxious today. Anxiety can be challenging to manage, but there are strategies that can help. Have you tried any relaxation techniques, such as deep breathing exercises or mindfulness meditation, to help calm your mind? Taking breaks throughout the day to engage in activities you enjoy or practicing self-care can also be beneficial. Additionally, talking to a trusted friend, family member, or professional counselor about your feelings may provide you with some support. Remember that it's okay to seek help when needed, and you don't have to go through this alone.

## User Input

- **Input**: I'm so angry right now, everything is frustrating and it feels like no one understands me.

## Sentiment Amplitude

- **Amplitude**: 0.3976190476190476

## Quantum State: [2.46733387e-33 1.28206558e-33 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.41938915e-01 6.58061085e-01]
## Compassion Scenario

I'm sorry to hear that you're feeling angry and frustrated. It can be difficult when it seems like no one understands what you're going through. Would you like to talk about what's been bothering you? Sometimes just expressing your feelings can help alleviate some of the anger and frustration.

## User Input

- **Input**: I feel so lonely, it's like I'm surrounded by people but no one really sees me or understands.

## Sentiment Amplitude

- **Amplitude**: 0.525

## Quantum State: [0.1124612  0.13161087 0.07898624 0.06749356 0.04050625 0.04740357
 0.07898624 0.06749356 0.04050625 0.04740357 0.02844924 0.02430982
 0.04050625 0.04740357 0.07898624 0.06749356]
## Compassion Scenario

I'm sorry to hear that you're feeling lonely. It can be difficult when you feel like you're surrounded by people but still don't feel seen or understood. Remember that you're not alone in experiencing these emotions, and there are steps you can take to improve your situation. Building meaningful connections and seeking understanding can often help alleviate feelings of loneliness. Is there anything specific you'd like to talk about or any suggestions you're looking for?

## User Input

- **Input**: I feel so betrayed and hurt, it's hard to believe that someone I trusted could do this.

## Sentiment Amplitude

- **Amplitude**: 0.35416666666666663

## Quantum State: [0.35835107 0.13856896 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.1402867  0.36279328]
## Compassion Scenario

I'm sorry to hear that you're feeling betrayed and hurt. It can be incredibly difficult to process when someone we trust lets us down. It's important to give yourself time and space to heal from these emotions. If you're comfortable sharing, I'm here to listen and offer any support or advice you may need.

## User Input

- **Input**: I'm scared about what the future holds, there's so much uncertainty and it's overwhelming.

## Sentiment Amplitude

- **Amplitude**: 0.6166666666666667

## Quantum State: [1.20286743e-33 2.54653203e-33 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 6.79183975e-01 3.20816025e-01]
## Compassion Scenario

I understand that feeling overwhelmed by uncertainty can be quite scary. It's not uncommon to have concerns about the future, especially when there are so many unknown variables. However, it's important to remember that uncertainty is a natural part of life, and many people feel this way at times.

One way to address this fear is by focusing on the things within your control. By taking proactive steps, such as setting goals, making plans, and taking care of your well-being, you can regain a sense of stability and empowerment. It can also be helpful to remind yourself of past times when you successfully navigated uncertainty, as this can boost your confidence.

Additionally, seeking support from loved ones or professionals can provide a safe space to express your concerns and gain new perspectives. Remember that you are not alone in feeling uncertain about the future, and there are resources available to help you navigate these emotions.

Lastly, practicing self-care and mindfulness techniques can help reduce anxiety and promote a more balanced mindset. Engaging in activities that bring you joy, finding moments of relaxation, and focusing on the present moment can all contribute to a greater sense of calm amidst uncertainty.

Remember, while the future may be uncertain, you have the strength and resilience to face whatever comes your way.